In [45]:
#importing the required libraries
from keras.models import Sequential
from keras import layers
import keras
import numpy as np
import random
import sys
import string

In [2]:
#loading the data ; source:https://www.gutenberg.org/ebooks/1342
text = open(r'pride.txt', encoding="utf8")
text = text.read().lower()
print('lenght: ', len(text))

lenght:  703829


In [3]:
#Just removed the starting useless part of the file
text=text[625:]

In [4]:
text[:1000]

'pride and prejudice\n\nby jane austen\n\n\n\nchapter 1\n\n\nit is a truth universally acknowledged, that a single man in possession\nof a good fortune, must be in want of a wife.\n\nhowever little known the feelings or views of such a man may be on his\nfirst entering a neighbourhood, this truth is so well fixed in the minds\nof the surrounding families, that he is considered the rightful property\nof some one or other of their daughters.\n\n“my dear mr. bennet,” said his lady to him one day, “have you heard that\nnetherfield park is let at last?”\n\nmr. bennet replied that he had not.\n\n“but it is,” returned she; “for mrs. long has just been here, and she\ntold me all about it.”\n\nmr. bennet made no answer.\n\n“do you not want to know who has taken it?” cried his wife impatiently.\n\n“_you_ want to tell me, and i have no objection to hearing it.”\n\nthis was invitation enough.\n\n“why, my dear, you must know, mrs. long says that netherfield is taken\nby a young man of large fortune

## Character level text generation using language model


In [5]:
maxlen = 60 # extracting sequence of 60 characters
step = 3 # sampling a new sequence every 3 characters
sentences = [] # holds the extracted seq
next_chars = [] # holds the targets(follow up char)

for i in range(0, len(text)- maxlen,step):
    sentences.append(text[i:i+maxlen])
    next_chars.append(text[i+maxlen])
print('number of sequences:', len(sentences))

number of sequences: 234382


In [6]:
#this is how it looks
sentences[:10]

['pride and prejudice\n\nby jane austen\n\n\n\nchapter 1\n\n\nit is a t',
 'de and prejudice\n\nby jane austen\n\n\n\nchapter 1\n\n\nit is a trut',
 'and prejudice\n\nby jane austen\n\n\n\nchapter 1\n\n\nit is a truth u',
 ' prejudice\n\nby jane austen\n\n\n\nchapter 1\n\n\nit is a truth univ',
 'ejudice\n\nby jane austen\n\n\n\nchapter 1\n\n\nit is a truth univers',
 'dice\n\nby jane austen\n\n\n\nchapter 1\n\n\nit is a truth universall',
 'e\n\nby jane austen\n\n\n\nchapter 1\n\n\nit is a truth universally a',
 'by jane austen\n\n\n\nchapter 1\n\n\nit is a truth universally ackn',
 'jane austen\n\n\n\nchapter 1\n\n\nit is a truth universally acknowl',
 'e austen\n\n\n\nchapter 1\n\n\nit is a truth universally acknowledg']

In [7]:
next_chars[:10]

['r', 'h', 'n', 'e', 'a', 'y', 'c', 'o', 'e', 'e']

In [8]:
chars= sorted(list(set(text))) #holds the unique chars from the corpus
print('Unique chars :',len(chars))
char_indices= dict((char,chars.index(char)) for char in chars) #Dictionary that maps unique characters to their index in the list “chars”

Unique chars : 56


In [9]:
#vectorization
#one-hot encoding the chars into the binary arrays
x=np.zeros((len(sentences),maxlen,len(chars)),dtype=np.bool)
y=np.zeros((len(sentences),len(chars)), dtype=np.bool)
for i,sentence in enumerate(sentences):
    for t,char in enumerate(sentence):
        x[i,t,char_indices[char]]=1
    y[i,char_indices[next_chars[i]]]=1

In [16]:
#building the network
model = Sequential()
model.add(layers.LSTM(128,input_shape=(maxlen,len(chars))))
model.add(layers.Dense(len(chars),activation='softmax'))

In [17]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy',optimizer=optimizer)

In [12]:
#function to sample the next char given the model's predictions
def sample(preds,temp=1.0):
    preds =np.asarray(preds).astype('float64')
    preds =np.log(preds)/temp
    exp_preds =np.exp(preds)
    preds = exp_preds/np.sum(exp_preds)
    probas =np.random.multinomial(1,preds,1)
    return np.argmax(probas)

In [18]:
#text generation 
for epoch in range(1,40): #training the model for 60 epochs
    model.fit(x,y,batch_size=128,epochs=1) #fit the model
    if(epoch == 20 or epoch == 39):
        print('epoch: ' ,epoch)
        #select a seed at random
        start_index = random.randint(0,len(text)-maxlen-1)
        generated_text = text[start_index: start_index+ maxlen]
        print('\n---Generating with seed: "'+ generated_text + '"')
        seed_text=generated_text
        #using different temperatures
        for temp in [0.2,0.5,1.0]:
            print('\n-------temperature: ',temp)
            sys.stdout.write(seed_text)
            for i in range(200): #generating 200 chars
                #one hot encode the chars generated so far
                sampled = np.zeros((1,maxlen,len(chars)))
                for t,char in enumerate(generated_text):
                    sampled[0,t,char_indices[char]] =1 
                
                #samples the next char
                preds= model.predict(sampled, verbose=0)[0]
                next_index = sample(preds,temp)
                next_char = chars[next_index]
                
                generated_text += next_char
                generated_text = generated_text[1:]
                sys.stdout.write(next_char)

Epoch 1/1
234382/234382 [==============================] - 263s 1ms/step - loss: 1.8468
Epoch 1/1
234382/234382 [==============================] - 259s 1ms/step - loss: 1.4852
Epoch 1/1
234382/234382 [==============================] - 259s 1ms/step - loss: 1.4014
Epoch 1/1
234382/234382 [==============================] - 257s 1ms/step - loss: 1.3610
Epoch 1/1
234382/234382 [==============================] - 263s 1ms/step - loss: 1.3327
Epoch 1/1
234382/234382 [==============================] - 262s 1ms/step - loss: 1.3119
Epoch 1/1
234382/234382 [==============================] - 258s 1ms/step - loss: 1.2958
Epoch 1/1
234382/234382 [==============================] - 257s 1ms/step - loss: 1.2847
Epoch 1/1
234382/234382 [==============================] - 259s 1ms/step - loss: 1.2743
Epoch 1/1
234382/234382 [==============================] - 257s 1ms/step - loss: 1.2629
Epoch 1/1
234382/234382 [==============================] - 258s 1ms/step - loss: 1.2569
Epoch 1/1
234382/234382 [=======

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


miss binglind timest, “and he is conn”““efeecal,
had
srighted at ne fully of the company acquaintance, satiness be suspiciess which had, unconmentment of she hastear
parties not capimance which i

## Word level text generation using language model


In [82]:
#lets do some preprocessing
#replace '-' with ' '
doc=text.replace('-',' ')
#split into tokens by white space
tokens=doc.split() 
#remove punctuation from each token
table =str.maketrans('','',string.punctuation)
tokens =[w.translate(table) for w in tokens]
#remove remaining tokens that are not alphabetic
tokens =[word for word in tokens if word.isalpha()]
print('Total tokens :',len(tokens))
print('Unique tokens :',len(set(tokens)))

Total tokens : 121488
Unique tokens : 6513


In [85]:
maxlen = 30 # extracting sequence of 30 words
step = 3 # sampling a new sequence every 3 words
sentences = [] # holds the extracted seq
next_words = [] # holds the targets(follow up words)

for i in range(0, len(tokens)- maxlen,step):
    sentences.append(' '.join(tokens[i:i+maxlen]))
    next_words.append(tokens[i+maxlen])
print('number of sequences:', len(sentences))

number of sequences: 40486


In [86]:
sentences[:10]

['pride and prejudice by jane austen chapter it is a truth universally acknowledged that a single man in possession of a good fortune must be in want of a wife',
 'by jane austen chapter it is a truth universally acknowledged that a single man in possession of a good fortune must be in want of a wife however little known',
 'chapter it is a truth universally acknowledged that a single man in possession of a good fortune must be in want of a wife however little known the feelings or',
 'a truth universally acknowledged that a single man in possession of a good fortune must be in want of a wife however little known the feelings or views of such',
 'acknowledged that a single man in possession of a good fortune must be in want of a wife however little known the feelings or views of such a man may',
 'single man in possession of a good fortune must be in want of a wife however little known the feelings or views of such a man may be on his',
 'possession of a good fortune must be in want of

In [87]:
next_words[:10]

['however',
 'the',
 'views',
 'a',
 'be',
 'first',
 'neighbourhood',
 'is',
 'fixed',
 'minds']

In [88]:
words= sorted(list(set(tokens))) #holds the unique words from the corpus
print('Unique chars :',len(words))
word_indices= dict((word,words.index(word)) for word in words) #Dictionary that maps unique word to their index in the list “word”

Unique chars : 6513


In [89]:
#vectorization
#one-hot encoding the tokens into the binary arrays
x=np.zeros((len(sentences),maxlen,len(words)),dtype=np.bool)
y=np.zeros((len(sentences),len(words)), dtype=np.bool)
for i,sentence in enumerate(sentences):
    for t,word in enumerate(sentence.split()):
        x[i,t,word_indices[word]]=1
    y[i,word_indices[next_words[i]]]=1

In [163]:
#building the network
model = Sequential()
model.add(layers.LSTM(128,input_shape=(maxlen,len(words))))
model.add(layers.Dense(128,activation='relu'))
model.add(layers.Dense(len(words),activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 128)               3400704   
_________________________________________________________________
dense_9 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_10 (Dense)             (None, 6513)              840177    
Total params: 4,257,393
Trainable params: 4,257,393
Non-trainable params: 0
_________________________________________________________________


In [164]:
optimizer = keras.optimizers.adam(lr=0.01)
model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])

In [92]:
#function to sample the next word given the model's predictions
def sample(preds,temp=1.0):
    preds =np.asarray(preds).astype('float64')
    preds =np.log(preds)/temp
    exp_preds =np.exp(preds)
    preds = exp_preds/np.sum(exp_preds)
    probas =np.random.multinomial(1,preds,1)
    return np.argmax(probas)

In [165]:
#text generation 
for epoch in range(1,30): #training the model for 30 epochs
    model.fit(x,y,batch_size=128,epochs=1) #fit the model
    if(epoch == 15 or epoch == 29):
        print('epoch: ' ,epoch)
        #select a seed at random
        start_index = random.randint(0,len(tokens)-maxlen-1)
        generated_text = tokens[start_index: start_index+ maxlen]
        generated_text = ' '.join(generated_text)
        print('\n---Generating with seed: "'+ generated_text + '"')
        seed_text=generated_text
        #using different temperatures
        for temp in [0.2,0.5,1.0]:
            print('\n-------temperature: ',temp)
            sys.stdout.write(seed_text)
            for i in range(45): #generating 45 words
                #one hot encode the words generated so far
                sampled = np.zeros((1,maxlen,len(words)))
                for t,word in enumerate(generated_text.split()):
                    sampled[0,t,word_indices[word]] =1 
                
                #samples the next word
                preds= model.predict(sampled, verbose=0)[0]
                next_index = sample(preds,temp)
                next_word = words[next_index]
                
                generated_text =generated_text+' '+ next_word
                generated_text=' '.join(generated_text.split()[1:])
                sys.stdout.write(' '+next_word)

Epoch 1/1
40486/40486 [==============================] - 208s 5ms/step - loss: 6.4778 - acc: 0.0354
Epoch 1/1
40486/40486 [==============================] - 178s 4ms/step - loss: 5.9264 - acc: 0.0717
Epoch 1/1
40486/40486 [==============================] - 203s 5ms/step - loss: 5.5389 - acc: 0.1021
Epoch 1/1
40486/40486 [==============================] - 211s 5ms/step - loss: 5.2385 - acc: 0.1295
Epoch 1/1
40486/40486 [==============================] - 241s 6ms/step - loss: 4.9829 - acc: 0.1492
Epoch 1/1
40486/40486 [==============================] - 266s 7ms/step - loss: 4.7457 - acc: 0.1686
Epoch 1/1
40486/40486 [==============================] - 230s 6ms/step - loss: 4.5305 - acc: 0.1900
Epoch 1/1
40486/40486 [==============================] - 239s 6ms/step - loss: 4.3198 - acc: 0.2115
Epoch 1/1
40486/40486 [==============================] - 219s 5ms/step - loss: 4.1216 - acc: 0.2369
Epoch 1/1
40486/40486 [==============================] - 223s 6ms/step - loss: 3.9256 - acc: 0.2625


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


 it is a very good room had soon at length when she had felt that i am quite many friend when the next room before their father were sought on these sort of three
-------temperature:  0.5
meryton was within a walk of longbourn they would be going there forever anxiety on janes behalf was another prevailing concern and mr darcys explanation by restoring bingley to all other elopement to myself they will be so objection to yourself what feeling a come to his sisters fatigued to the mortifying of his words that recommend himself by those but elizabeth had never been to do one and then there is not a step
-------temperature:  1.0
meryton was within a walk of longbourn they would be going there forever anxiety on janes behalf was another prevailing concern and mr darcys explanation by restoring bingley to all directly in another two wickham was what i know in leaving before one or both or every day she had not all spent in such other elizabeth can animated that to receive him than madam you

* Seems like the results are best at temp=0.2 .